In [61]:
import glob, json, os, re

In [66]:
!ls -lh columncache/*/*
!grep -i 'technical doc' columncache/*/description.html

-rw-rw-r--  1 rsargent  staff   3.5M Mar  6  2018 columncache/acs2009_5year_tract2010/description.html
-rw-rw-r--  1 rsargent  staff   5.7M Feb 23  2018 columncache/acs2010_5year_tract2010/description.html
-rw-rw-r--  1 rsargent  staff   3.5M Feb 21  2018 columncache/acs2011_5year_tract2010/description.html
-rw-rw-r--  1 rsargent  staff   2.9M Feb 21  2018 columncache/acs2012_5year_tract2010/description.html
-rw-rw-r--  1 rsargent  staff   3.8M Feb 20  2018 columncache/acs2013_5year_tract2010/description.html
-rw-r--r--  1 rsargent  staff   1.6M Dec 28 06:26 columncache/acs2013_5year_tract2010/description.json
-rw-rw-r--  1 rsargent  staff   4.8M Feb 14  2018 columncache/acs2014_5year_tract2010/description.html
-rw-r--r--  1 rsargent  staff   1.6M Dec 28 06:26 columncache/acs2014_5year_tract2010/description.json
-rw-rw-r--  1 rsargent  staff   6.0M Sep  1  2017 columncache/acs2015_5year_tract2010/description.html
-rw-rw-r--  1 rsargent  staff   6.1M Feb 14  2018 columncache/acs2016_5ye

In [56]:
desc_htmls = glob.glob('columncache/*/description.html')
desc_htmls

['columncache/acs2013_5year_tract2010/description.html',
 'columncache/acs2014_5year_tract2010/description.html',
 'columncache/census1990_block2010/description.html',
 'columncache/acs2015_5year_tract2010/description.html',
 'columncache/census2000_block2010/description.html',
 'columncache/acs2012_5year_tract2010/description.html',
 'columncache/acs2010_5year_tract2010/description.html',
 'columncache/acs2009_5year_tract2010/description.html',
 'columncache/census2010_block2010/description.html',
 'columncache/acs2011_5year_tract2010/description.html',
 'columncache/acs2016_5year_tract2010/description.html']

In [77]:
def desc2json(desc_html):
    desc_json = desc_html.replace('.html', '.json')
    #if os.path.exists(desc_json):
    #    print('%s already exists, skipping' % desc_json)
    #    return
    html = open(desc_html).read()
    lines = html.split("\n")
    print('Read %d lines from %s' % (len(lines), desc_html))

    j = {'tables':[]}

    prev_table_name = None

    for line in lines:
        doc = re.search(r'<a href="(.*)">(.*)</a>', line)
        if doc:
            j['docs'] = [{'url': doc.groups()[0], 'desc': doc.groups()[1]}]
            continue
        match = re.match(r'<tr><td>(.*)</td><td>(.*)</td></tr>', line)
        if not match:
            continue
        (name, desc) = match.groups()
        if not '.' in name:
            name = os.path.basename(os.path.dirname(desc_html)) + '.' + name
        (dataset_name, full_col_name) = name.split('.')
        if '_' in full_col_name:
            (table_name, col_name) = full_col_name.split('_')
        elif len(full_col_name) == 9:
            table_name = full_col_name[:5]
            col_name = full_col_name[5:]
        elif len(full_col_name) == 8:
            table_name = full_col_name[:4]
            col_name = full_col_name[4:]
        elif len(full_col_name) == 7:
            table_name = full_col_name[:4]
            col_name = full_col_name[4:]
        else:
            print(full_col_name)
            
        if not '&mdash;' in desc:
            desc += ' &mdash; Total'
        (table_desc, col_desc) = [x.strip() for x in desc.split('&mdash;', 1)]
        if table_name != prev_table_name:
            j['name'] = dataset_name
            j['tables'].append({'name': table_name, 'description': table_desc, 'columns': []})
            prev_table_name = table_name
        j['tables'][-1]['columns'].append([col_name, col_desc])

    print(len(json.dumps(j)), len(html))

    json.dump(j, open(desc_json, 'w'))
    print('Created %s' % desc_json)

for desc_html in desc_htmls:
    desc2json(desc_html)


Read 22712 lines from columncache/acs2013_5year_tract2010/description.html
1633519 3934993
Created columncache/acs2013_5year_tract2010/description.json
Read 22628 lines from columncache/acs2014_5year_tract2010/description.html
1728496 5079642
Created columncache/acs2014_5year_tract2010/description.json
Read 986 lines from columncache/census1990_block2010/description.html
47289 89287
Created columncache/census1990_block2010/description.json
Read 22768 lines from columncache/acs2015_5year_tract2010/description.html
1792690 6337359
Created columncache/acs2015_5year_tract2010/description.json
Read 3064 lines from columncache/census2000_block2010/description.html
226484 533817
Created columncache/census2000_block2010/description.json
Read 22528 lines from columncache/acs2012_5year_tract2010/description.html
1618750 2999775
Created columncache/acs2012_5year_tract2010/description.json
Read 21488 lines from columncache/acs2010_5year_tract2010/description.html
1687046 5992902
Created columncach

In [ ]:
rsync -av --exclude="*.html" columncache hal15:/home/rsargent/uwsgi/dotmaptiles-data/server
